In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-13 10:45:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    in 0,02s   

2023-03-13 10:45:11 (56,2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [23]:
# open the text file and read in data as `text`
with open('input.txt', 'r', encoding="utf-8") as f:
    text = f.read()


In [24]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [25]:
print("length of the text is {} characters".format(len(text)))

length of the text is 1115394 characters


In [26]:
# get the unique characters in the file
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

print("There are {} unique characters in the file".format(vocab_size))
print(vocab)

There are 65 unique characters in the file
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [27]:
# create a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = {i:u for i, u in enumerate(vocab)}

encode = lambda x: [char2idx[c] for c in x]
decode = lambda x: ''.join([idx2char[c] for c in x])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [28]:
# encode the text
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [29]:
# split the dataset into train and test
train_size = int(0.9 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

In [30]:
print(train_data.shape)
print(val_data.shape)

torch.Size([1003854])
torch.Size([111540])


In [31]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [32]:
# sample a sequence of length `block_size` from the dataset
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when context is: {context}, target is: {target}")

when context is: tensor([18]), target is: 47
when context is: tensor([18, 47]), target is: 56
when context is: tensor([18, 47, 56]), target is: 57
when context is: tensor([18, 47, 56, 57]), target is: 58
when context is: tensor([18, 47, 56, 57, 58]), target is: 1
when context is: tensor([18, 47, 56, 57, 58,  1]), target is: 15
when context is: tensor([18, 47, 56, 57, 58,  1, 15]), target is: 47
when context is: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is: 58


In [35]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # randomly select the starting indices for the examples in the mini-batch
    start_indices = torch.randint(len(data) - block_size, size=(batch_size,))
    # select the contiguous sequences of tokens
    x = torch.stack([data[i:i+block_size] for i in start_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in start_indices])
    return x, y

In [36]:
xb, yb = get_batch('train')
print("input shape: ", xb.shape)
print("target shape: ", yb.shape)
print(xb)
print(yb)

input shape:  torch.Size([4, 8])
target shape:  torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [61]:

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.emb(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            logits = logits.view(-1, logits.shape[-1]) # (B * T, C)
            targets = targets.view(-1) # (B * T)
            loss  = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1)
            new_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, new_idx], dim=1) # (B, T+1)
        return idx


In [57]:
model = BigramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print(out.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [63]:
torch.manual_seed(1337)
idx = torch.zeros((1, 1), dtype=torch.long)
generated = model.generate(idx, max_new_tokens=100)
decoded = decode(generated[0].tolist())
print(decoded)


CpzlYaS CczdeC'aEHY,YzrzPGJU?YehOLi
Cj
CzoX$kUtSqAUU'zPyVJiG HTosttx;a!UVzyAnsz-imszPAkfya;:gU;RTNBG


In [64]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [72]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print(f"step: {steps}, loss: {loss.item():.3f}")

print("Training done!")
print(loss.item())   



step: 0, loss: 2.472
step: 100, loss: 2.584
step: 200, loss: 2.516
step: 300, loss: 2.442
step: 400, loss: 2.520
step: 500, loss: 2.423
step: 600, loss: 2.418
step: 700, loss: 2.440
step: 800, loss: 2.449
step: 900, loss: 2.427
step: 1000, loss: 2.510
step: 1100, loss: 2.479
step: 1200, loss: 2.378
step: 1300, loss: 2.490
step: 1400, loss: 2.593
step: 1500, loss: 2.407
step: 1600, loss: 2.424
step: 1700, loss: 2.426
step: 1800, loss: 2.292
step: 1900, loss: 2.653
step: 2000, loss: 2.417
step: 2100, loss: 2.582
step: 2200, loss: 2.465
step: 2300, loss: 2.328
step: 2400, loss: 2.476
step: 2500, loss: 2.277
step: 2600, loss: 2.533
step: 2700, loss: 2.396
step: 2800, loss: 2.495
step: 2900, loss: 2.360
step: 3000, loss: 2.424
step: 3100, loss: 2.544
step: 3200, loss: 2.556
step: 3300, loss: 2.485
step: 3400, loss: 2.396
step: 3500, loss: 2.366
step: 3600, loss: 2.517
step: 3700, loss: 2.367
step: 3800, loss: 2.405
step: 3900, loss: 2.415
step: 4000, loss: 2.446
step: 4100, loss: 2.455
step

In [73]:
torch.manual_seed(1337)
idx = torch.zeros((1, 1), dtype=torch.long)
generated = model.generate(idx, max_new_tokens=500)
decoded = decode(generated[0].tolist())
print(decoded)


Cars an tcede.
Y in, lanoul-see waindonse ate t,-bee wist ic wsoster; bea yonsenimser se ay g pourancey mou ber s LI'sl tem'ls tofr?

KESod, IAg thorvere nonifit deanche
We trerath; shan ise pls tode ish rithoo bsin henghimblyondisotinoes selonsen l'se ph POUMAnome, ty V: gill.
Shly a au a owe hotemanven.
Y:
HAnouro astroptong,
Breavithiritoull howie ssu hathitious gh healol lemek'le. shily ss tiswo amanghen ivethe:
IOf chawh if cloomace ifth reris r blithout yoe n,
T:

TIUSpl sur t kechug t, nt
